In [1]:
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0


In [2]:
#LOAD DEPENDENCIES
import os
import cv2
import time
import pickle
import logging
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.utils import class_weight
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras import applications
from tensorflow.keras import Model, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import AveragePooling2D, AlphaDropout, Activation, Add, BatchNormalization, Concatenate, Layer, ReLU, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout,Flatten

from tensorflow.keras.applications.efficientnet import EfficientNetB0 as trainable_model_a
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 as trainable_model_b
from tensorflow.keras.applications.resnet_v2 import ResNet50V2 as trainable_model_c

from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

#PREVENT ERROR UNCESSARY MESSAGES
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [5]:
#LOAD THE DATA
# Training Data split 70% training and 30% validation

train_data_dir = "/content/drive/MyDrive/Rice Leaf Diseases/Rice leaf (70% 20% 10%)/Test/train"
validation_data_dir = "/content/drive/MyDrive/Rice Leaf Diseases/Rice leaf (70% 20% 10%)/Test/val"
test_data_dir = "/content/drive/MyDrive/Rice Leaf Diseases/Rice leaf (70% 20% 10%)/Test/test"


# THE INPUT LAYER IS THE SAME AS IT WILL BE FUSED AS ONE LATER ON
img_rows, img_cols = 224, 224
input_shape = (img_rows,img_cols,3)
model_input = Input(shape=input_shape)
print("Data folders found!")
print("The Input size is set to ", model_input)

Data folders found!
The Input size is set to  KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_2'), name='input_2', description="created by layer 'input_2'")


In [6]:
from keras.preprocessing.image import ImageDataGenerator
batch_size = 16
# Define data generator with desired transformations
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest')
val_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest')
test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows,img_cols),
        batch_size=batch_size,
        class_mode='categorical',
         classes=['Bacterialblight', 'Blast','Brownspot','Tungro'])

validation_generator = val_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows,img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False,
        classes=['Bacterialblight', 'Blast','Brownspot','Tungro'])

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_rows,img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        classes=['Bacterialblight', 'Blast','Brownspot','Tungro'])

#CHECK  THE NUMBER OF SAMPLES
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

if nb_train_samples == 0:
    print("NO DATA TRAIN FOUND! Please check your train data path and folders!")
else:
    print("Train samples found!")

if nb_validation_samples == 0:
    print("NO DATA VALIDATION FOUND! Please check your validation data path and folders!")
    print("Check the data folders first!")
else:
    print("Validation samples found!")

if nb_test_samples == 0:
    print("NO DATA VALIDATION FOUND! Please check your validation data path and folders!")
    print("Check the data folders first!")
else:
    print("Test samples found!")



#check the class indices
train_generator.class_indices
validation_generator.class_indices
test_generator.class_indices

#true labels
Y_test=validation_generator.classes

num_classes= len(train_generator.class_indices)

if nb_train_samples and nb_validation_samples > 0:
    print("Generators are set!")
    print("Check if dataset is complete and has no problems before proceeding.")

classnames = list(train_generator.class_indices.keys())
print("class names: ", classnames)

Found 4150 images belonging to 4 classes.
Found 1185 images belonging to 4 classes.
Found 597 images belonging to 4 classes.
Train samples found!
Validation samples found!
Test samples found!
Generators are set!
Check if dataset is complete and has no problems before proceeding.
class names:  ['Bacterialblight', 'Blast', 'Brownspot', 'Tungro']


In [8]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(class_weight = "balanced", classes= np.unique(train_generator.classes), y= train_generator.classes)
print(class_weights)

[0.93637184 1.03028798 0.92633929 1.13387978]


In [10]:
#Set constants
batch_size = 16
epochs = 50
optimizer = Adam(learning_rate=0.0001)
architecture = 'Ensemble'

DCNN_A = 'DCNN_A'
DCNN_B = 'DCNN_B'
DCNN_C = 'DCNN_C'

In [11]:
# Model
#EfficientNetB0

builder_a = DCNN_A + '_builder'

#TRANSFER LEARNING
def builder_a(model_input):
    builder_a = trainable_model_a(weights='imagenet',
                                    include_top=False,
                                    input_tensor = model_input)

#PARTIAL LAYER FREEZING
    for layer in builder_a.layers:
        layer.trainable = False

    for layer in builder_a.layers:
        layer._name = layer.name + '_' + DCNN_A

    for BatchNormalization in builder_a.layers:
        BatchNormalization.trainable = False

    x = builder_a.layers[-17].output

#AUXILIARY FUSING LAYER (AuxFL)
    x = Conv2D(192, 1, padding='valid', activation='selu', kernel_initializer='lecun_normal')(x)
    x = AveragePooling2D(1, 1)(x)

    dcnn_a = Model(inputs=builder_a.input, outputs=x, name=DCNN_A)
    return dcnn_a

#INITIALIZE THE MODEL
dcnn_a = builder_a(model_input)

#PLOT THE MODEL STRUCTURE
print("PLEASE CHECK THE ENTIRE MODEL UP TO THE END")

dcnn_a.summary()

print("successfully built!")

16705208/16705208 [==============================] - 0s 0us/step
PLEASE CHECK THE ENTIRE MODEL UP TO THE END
Model: "DCNN_A"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2_DCNN_A (InputLayer  [(None, 224, 224, 3)]        0         []                            
 )                                                                                                
                                                                                                  
 rescaling_DCNN_A (Rescalin  (None, 224, 224, 3)          0         ['input_2_DCNN_A[0][0]']      
 g)                                                                                               
                                                                                                  
 normalization_DCNN_A (Norm  (None, 224, 224, 3)          7         ['rescaling_DCN

In [12]:
# Model
#MobileNetV2

builder_b = DCNN_B + '_builder'

#TRANSFER LEARNING
def builder_b(model_input):
    builder_b = trainable_model_b(weights='imagenet',
                                    include_top=False,
                                    input_tensor = model_input)

#PARTIAL LAYER FREEZING
    for layer in builder_b.layers:
        layer.trainable = False

    for layer in builder_b.layers:
        layer._name = layer.name + '_' + DCNN_B

    for BatchNormalization in builder_b.layers:
        BatchNormalization.trainable = False

    x = builder_b.layers[-39].output

#AUXILIARY FUSING LAYER (AuxFL)
    x = Conv2D(192, 8, padding='valid', activation='selu', kernel_initializer='lecun_normal')(x)
    x = AveragePooling2D(1, 1)(x)

    dcnn_b = Model(inputs=builder_b.input, outputs=x, name=DCNN_B)
    return dcnn_b

#INITIALIZE THE MODEL
dcnn_b = builder_b(model_input)

#PLOT THE MODEL STRUCTURE
print("PLEASE CHECK THE ENTIRE MODEL UP TO THE END")

dcnn_b.summary()

print(" successfully built!")

9406464/9406464 [==============================] - 0s 0us/step
PLEASE CHECK THE ENTIRE MODEL UP TO THE END
Model: "DCNN_B"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2_DCNN_A_DCNN_B (Inp  [(None, 224, 224, 3)]        0         []                            
 utLayer)                                                                                         
                                                                                                  
 Conv1_DCNN_B (Conv2D)       (None, 112, 112, 32)         864       ['input_2_DCNN_A_DCNN_B[0][0]'
                                                                    ]                             
                                                                                                  
 bn_Conv1_DCNN_B (BatchNorm  (None, 112, 112, 32)         128       ['Conv1_DCNN_B[0]

In [13]:
# Model
#ResNet50V2

builder_c = DCNN_C + '_builder'

#TRANSFER LEARNING
def builder_c(model_input):
    builder_c = trainable_model_c(weights='imagenet',
                                    include_top=False,
                                    input_tensor = model_input)

#PARTIAL LAYER FREEZING
    for layer in builder_c.layers:
        layer.trainable = False

    for layer in builder_c.layers:
        layer._name = layer.name + '_' + DCNN_C

    for BatchNormalization in builder_c.layers:
        BatchNormalization.trainable = False

    x = builder_c.layers[-117].output

#AUXILIARY FUSING LAYER (AuxFL)
    x = Conv2D(192, 6, padding='valid', activation='selu', kernel_initializer='lecun_normal')(x)
    x = AveragePooling2D(3, 3)(x)

    dcnn_c = Model(inputs=builder_c.input, outputs=x, name=DCNN_C)
    return dcnn_c

#INITIALIZE THE MODEL
dcnn_c = builder_c(model_input)

#PLOT THE MODEL STRUCTURE
print("PLEASE CHECK THE ENTIRE MODEL UP TO THE END")

dcnn_c.summary()

print("successfully built!")

94668760/94668760 [==============================] - 1s 0us/step
PLEASE CHECK THE ENTIRE MODEL UP TO THE END
Model: "DCNN_C"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2_DCNN_A_DCNN_B_DCNN  [(None, 224, 224, 3)]        0         []                            
 _C (InputLayer)                                                                                  
                                                                                                  
 conv1_pad_DCNN_C (ZeroPadd  (None, 230, 230, 3)          0         ['input_2_DCNN_A_DCNN_B_DCNN_C
 ing2D)                                                             [0][0]']                      
                                                                                                  
 conv1_conv_DCNN_C (Conv2D)  (None, 112, 112, 64)         9472      ['conv1_pad_DCN

In [14]:
#RE-INITIALIZE FOR FUSION
dcnn_a = builder_a(model_input)
dcnn_b = builder_b(model_input)
dcnn_c = builder_c(model_input)

print("Accomplished Pre-training and ready for fusion")

Accomplished Pre-training and ready for fusion


In [15]:
#FUSE THE MODELS INTO A SINGLE PIPELINE

models = [dcnn_a,
          dcnn_b,
          dcnn_c]

print("Fusion success!")
print("Ready to connect with its ending layers!")

Fusion success!
Ready to connect with its ending layers!


In [16]:
#Build the Fusion Residual Block (FuRB)
def ensemble_builder(models, model_input):
    outputs = [m.output for m in models]

#INITIAL FUSION LAYER
    y = Add(name='InitialFusionLayer')(outputs)

#FuRB
    y_bn1 = BatchNormalization()(y)
    y_selu1 = tf.keras.activations.selu(y_bn1)
    y_conv1 = Conv2D(192, 1, kernel_initializer='lecun_normal')(y_selu1)

    y_bn2 = BatchNormalization()(y_conv1)
    y_selu2 = tf.keras.activations.selu(y_bn2)
    y_conv2 = Conv2D(192, 1, kernel_initializer='lecun_normal')(y_selu2)

    y_bn3 = BatchNormalization()(y_conv2)
    y_selu3 = tf.keras.activations.selu(y_bn3)
    y_conv3 = Conv2D(192, 1, kernel_initializer='lecun_normal')(y_selu3)

    y_bn4 = BatchNormalization()(y_conv3)
    y_selu4 = tf.keras.activations.selu(y_bn4)
    y_conv4 = Conv2D(192, 1, kernel_initializer='lecun_normal')(y_selu4)

    y_bn5 = BatchNormalization()(y_conv4)
    y_selu5 = tf.keras.activations.selu(y_bn5)
    y_conv5 = Conv2D(192, 1, kernel_initializer='lecun_normal')(y_selu5)

    y_merge = Add(name='ResidualFusionLayer')([y, y_conv5])

#FINE-TUNING
    y = GlobalAveragePooling2D()(y_merge)
    y = AlphaDropout(0.5)(y)
    prediction = Dense(num_classes,activation='softmax', name='Softmax_Classifier' + architecture)(y)
    model = Model(model_input, prediction, name=architecture)
    return model

#Istantitate the model and report the summary
Ensemble = ensemble_builder(models, model_input)

print()
print()
print("PLEASE CHECK THE MODEL UP TO THE END")
print()
print()
print()

Ensemble.summary()
print("Complete and ready for compilation and training!")



PLEASE CHECK THE MODEL UP TO THE END



Model: "Ensemble"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2_DCNN_A_DCNN_B_DCNN  [(None, 224, 224, 3)]        0         []                            
 _C_DCNN_A_DCNN_B_DCNN_C (I                                                                       
 nputLayer)                                                                                       
                                                                                                  
 rescaling_2_DCNN_A (Rescal  (None, 224, 224, 3)          0         ['input_2_DCNN_A_DCNN_B_DCNN_C
 ing)                                                               _DCNN_A_DCNN_B_DCNN_C[0][0]'] 
                                                                                                  
 normalization_1_DCNN_A (No  (None, 224, 224, 3) 

In [17]:
last_conv_layer_name =Ensemble.layers[-4].name

if last_conv_layer_name == 'ResidualFusionLayer':
    print("CORRECT LAYER SELECTED:", last_conv_layer_name)
else:
    print("INCORRECT LAYER SELECTED:", last_conv_layer_name)
    print("Please Reselect")

CORRECT LAYER SELECTED: ResidualFusionLayer


In [18]:
import tensorflow as tf

#MODEL COMPILATION WITH HYPER-PARAMETERS, LOSS FUNCTIONS AND TRAINING!
import time

batch_size = 16

epochs = 50

start_time = time.time()

optimizer = Adam(lr=0.0001)

Ensemble.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2,
                              verbose=1, mode='max', min_lr=0.000001)

callbacks = [reduce_lr]

# MODEL TRAINING

history = Ensemble.fit_generator(train_generator, steps_per_epoch=nb_train_samples // batch_size,
                                  epochs=epochs, validation_data=validation_generator,
                                  callbacks=callbacks,
                                  validation_steps=nb_validation_samples // batch_size, verbose=1)

elapsed_time = time.time() - start_time
train_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
print()
print()
print(train_time, 'train_time')
print()
print(elapsed_time, 'Seconds')
print()
print()
print("MODEL SERIALIZING WAIT FOR A MOMENT...")
print()

Ensemble.save('/content/drive/MyDrive/Rice Leaf Diseases/Rice leaf (70% 20% 10%)/Epoch 50 Batch size 16/Weights/model.h5')

from pickle import dump
dump(history, open('/content/drive/MyDrive/Rice Leaf Diseases/Rice leaf (70% 20% 10%)/Epoch 50 Batch size 16/History/model.pkl', 'wb'))

print()
print()
print("The Model weights and history are successfully trained and saved!")





<ipython-input-18-a21a936507cc>:23: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = Ensemble.fit_generator(train_generator, steps_per_epoch=nb_train_samples // batch_size,


Epoch 1/50
259/259 [==============================] - 1401s 5s/step - loss: 1.5845 - accuracy: 0.8004 - val_loss: 0.6033 - val_accuracy: 0.8834 - lr: 0.0010
Epoch 2/50
259/259 [==============================] - 123s 475ms/step - loss: 0.3255 - accuracy: 0.8965 - val_loss: 0.2006 - val_accuracy: 0.9595 - lr: 0.0010
Epoch 3/50
259/259 [==============================] - 102s 393ms/step - loss: 0.1980 - accuracy: 0.9354 - val_loss: 0.2071 - val_accuracy: 0.9628 - lr: 0.0010
Epoch 4/50
259/259 [==============================] - 102s 393ms/step - loss: 0.1651 - accuracy: 0.9448 - val_loss: 0.2946 - val_accuracy: 0.9476 - lr: 0.0010
Epoch 5/50
259/259 [==============================] - 101s 392ms/step - loss: 0.1464 - accuracy: 0.9536 - val_loss: 0.1448 - val_accuracy: 0.9747 - lr: 0.0010
Epoch 6/50
259/259 [==============================] - 102s 394ms/step - loss: 0.1290 - accuracy: 0.9538 - val_loss: 0.0654 - val_accuracy: 0.9848 - lr: 0.0010
Epoch 7/50
259/259 [============================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(




The Model weights and history are successfully trained and saved!
